In [5]:
import numpy as np
from aux_red_multi import generar_ventana, corpus_modificado, palabras_a_indice, indices_a_embeddings, W1
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
x_gen,y1,y2_one_hot =generar_ventana(corpus_modificado, palabras_a_indice, 10, indices_a_embeddings)

In [12]:
y1.max(), y1.min()

(13.169836067178666, -13.206317476270707)

In [ ]:
entradas = x_gen.shape[1]
salidas = W1.shape[0]

inputs = keras.Input(shape=(entradas,))

x = layers.Dense(500, activation='gelu')(inputs)

x = layers.Dense(600, activation='gelu')(x)

x = layers.Dense(1500, activation='gelu')(x)

outputs = layers.Dense(salidas, activation='softmax')(x)

multicapa_one_hot = keras.Model(inputs, outputs)

# X: embeddings concatenados o features
# Y: índice de palabra central

loss = keras.losses.SparseCategoricalCrossentropy(reduction="none")

multicapa_one_hot.compile(loss=loss, optimizer=keras.optimizers.Adam(1e-3))

multicapa_one_hot.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1300)]            0         
                                                                 
 dense (Dense)               (None, 500)               650500    
                                                                 
 dense_1 (Dense)             (None, 600)               300600    
                                                                 
 dense_2 (Dense)             (None, 1500)              901500    
                                                                 
 dense_3 (Dense)             (None, 7757)              11643257  
                                                                 
Total params: 13495857 (51.48 MB)
Trainable params: 13495857 (51.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

epocas = 5

multicapa_one_hot.fit(x_gen, y2_one_hot, 
                epochs= epocas,
                batch_size=50,
                shuffle=True,
                validation_split=0.2,
                callbacks=[early_stop])


Epoch 1/5

888/888 [==============================] - 145s 161ms/step - loss: 6.3479 - val_loss: 6.5886
Epoch 2/5
888/888 [==============================] - 164s 184ms/step - loss: 5.5755 - val_loss: 6.9336
Epoch 3/5
633/888 [====================>.........] - ETA: 41s - loss: 4.8822

KeyboardInterrupt: 

In [5]:
accuracy_top1, accuracy_topk, palabras_mal_predichas = predecir_cbow_completo_one_hot(palabras_a_indice, corpus_modificado, multicapa_one_hot, indices_a_embeddings, topk=5)


KeyboardInterrupt: 

In [ ]:
np.savez('pesos_autoencoder_gatos.npz', *multicapa_one_hot.get_weights())
with np.load('pesos_autoencoder_gatos.npz') as data:
    weights = [data[f'arr_{i}'] for i in range(len(data.files))]
multicapa_one_hot.set_weights(weights)

In [12]:
W1_min, W1_max = W1.min(), W1.max()

x_escalado = 2 * ((x_gen - W1_min) / (W1_max - W1_min)) - 1
y_escalado = 2 * ((y1 - W1_min) / (W1_max - W1_min)) - 1

In [10]:
entradas = x_gen.shape[1]
salidas = W1.shape[1]

inputs = keras.Input(shape=(entradas,))

x = layers.Dense(512, activation='gelu')(inputs)

x = layers.Dense(220, activation='gelu')(x)

outputs = layers.Dense(salidas, activation='tanh')(x)

multicapa_embeddings = keras.Model(inputs, outputs)

# X: embeddings concatenados o features
# Y: índice de palabra central
multicapa_embeddings.compile(loss='mse', optimizer=keras.optimizers.Adam(1e-3))

multicapa_embeddings.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1300)]            0         
                                                                 
 dense_9 (Dense)             (None, 512)               666112    
                                                                 
 dense_10 (Dense)            (None, 220)               112860    
                                                                 
 dense_11 (Dense)            (None, 130)               28730     
                                                                 
Total params: 807702 (3.08 MB)
Trainable params: 807702 (3.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
multicapa_embeddings.fit(x_escalado, y_escalado, 
                epochs=2000,
                batch_size=50,
                shuffle=True,
                validation_split=0.2)

Epoch 1/2000
4717/4717 [==============================] - 74s 15ms/step - loss: 0.0329 - val_loss: 0.0331
Epoch 2/2000
4717/4717 [==============================] - 65s 14ms/step - loss: 0.0327 - val_loss: 0.0331
Epoch 3/2000
4717/4717 [==============================] - 72s 15ms/step - loss: 0.0326 - val_loss: 0.0330
Epoch 4/2000
4717/4717 [==============================] - 65s 14ms/step - loss: 0.0326 - val_loss: 0.0330
Epoch 5/2000
3330/4717 [====================>.........] - ETA: 18s - loss: 0.0327

KeyboardInterrupt: 

In [6]:
# Cargar el modelo desde el archivo
modelo = keras.models.load_model("C:\\Users\\User\\Documents\\GitHub\\Aprendizaje_Automatico\\multicapa_onehot_model_epoca60.keras")

# Verificar la estructura del modelo
modelo.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1300)]            0         
                                                                 
 dense (Dense)               (None, 1024)              1332224   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 1024)              525312    
                                                                 
 dense_4 (Dense)             (None, 7757)              7950925   
                                                                 
Total params: 10595917 (40.42 MB)
Trainable params: 10595917

In [7]:
modelo.evaluate(x_gen, y2_one_hot)



1734/1734 [==============================] - 33s 18ms/step - loss: 0.4902 - accuracy: 0.8990


[0.49016594886779785, 0.8989975452423096]

In [8]:
# Cargar el modelo desde el archivo
modelo2 = keras.models.load_model("C:\\Users\\User\\Documents\\GitHub\\Aprendizaje_Automatico\\multicapa_emb_model_epoca.keras")

# Verificar la estructura del modelo
modelo2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1300)]            0         
                                                                 
 dense_6 (Dense)             (None, 1024)              1332224   
                                                                 
 dense_7 (Dense)             (None, 512)               524800    
                                                                 
 dense_8 (Dense)             (None, 512)               262656    
                                                                 
 dense_9 (Dense)             (None, 300)               153900    
                                                                 
 dense_10 (Dense)            (None, 130)               39130     
                                                                 
Total params: 2312710 (8.82 MB)
Trainable params: 2312710 (

In [10]:
modelo2.evaluate(x_gen, y1)

1734/1734 [==============================] - 8s 4ms/step - loss: 1.9121 - accuracy: 0.0127


[1.9121452569961548, 0.012728977017104626]